In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
from datetime import datetime

# ---------------------- 1. 数据读取与预处理（深圳数据） ----------------------
# 筛选深圳最新清洗数据（文件前缀含"shenzhen_cleaned_"）
filtered_files = [f for f in os.listdir(os.getcwd()) if f.startswith("pet_funeral_shenzhen_cleaned_20251115_001632")]
if not filtered_files:
    print("❌ No Shenzhen filtered files found!")
    exit()
latest_file = max(filtered_files, key=lambda x: os.path.getmtime(x))
df = pd.read_excel(latest_file)
total = len(df)
print(f"✅ Loaded data: {latest_file} (Total: {total} merchants in Shenzhen)")

# 提取有效价格+统计缺失数据（逻辑与北京完全一致）
def extract_valid_price(price_val):
    if pd.isna(price_val) or str(price_val).strip() in ['人均 -', '', 'None']:
        return None
    match = re.search(r'(\d+)', str(price_val))
    return float(match.group()) if match else None

df['valid_price'] = df['avg_price'].apply(extract_valid_price)
valid_data = df[df['valid_price'].notna()]['valid_price']
missing_count = total - len(valid_data)

# ---------------------- 2. 数据准备（价格区间与北京统一，便于跨城市对比） ----------------------
price_bins = [0, 500, 800, 1200, 1500, float('inf')]
price_labels = ['0-500 RMB', '501-800 RMB', '801-1200 RMB', '1201-1500 RMB', '>1500 RMB', 'Missing Data']
price_groups = pd.cut(valid_data, bins=price_bins, labels=price_labels[:-1], right=False)
price_counts = price_groups.value_counts().sort_index()
price_counts['Missing Data'] = missing_count

# ---------------------- 3. 绘制柱状图（紫色系渐变，深圳专属） ----------------------
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.family'] = ['Arial', 'Helvetica']
plt.rcParams['axes.unicode_minus'] = False

fig, ax = plt.subplots(figsize=(12, 7))
# 紫色系配色（从浅紫到深紫渐变，缺失数据用淡紫灰，共6色对应6个区间）
colors = [
    '#f3e5f5',  # 最浅紫（0-500 RMB）
    '#e1bee7',  # 浅紫（501-800 RMB）
    '#ce93d8',  # 中紫（801-1200 RMB）
    '#ba68c8',  # 深紫（1201-1500 RMB）
    '#9c27b0',  # 最深紫（>1500 RMB）
    '#e0d4e0'   # 淡紫灰（Missing Data）
]

bars = ax.bar(price_counts.index, price_counts.values, color=colors, width=0.65,
              edgecolor='white', linewidth=1.5, alpha=0.9)

# 数据标签（数量+百分比，格式与北京一致）
for bar in bars:
    height = bar.get_height()
    percentage = (height / total) * 100
    ax.text(bar.get_x() + bar.get_width()/2, height + 0.5,
            f"{int(height)}\n({percentage:.1f}%)",
            ha='center', va='bottom', fontsize=9, fontweight='bold', color='#333')

# 标题标注深圳，颜色与最深紫呼应
ax.set_title('Per Capita Price Distribution of Pet Funeral Merchants in Shenzhen',
             fontsize=16, fontweight='bold', color='#6a1b9a')  # 深紫标题色
ax.set_xlabel('Price Range', fontsize=12, fontweight='medium')
ax.set_ylabel('Number of Merchants', fontsize=12, fontweight='medium')

# 美化细节（与北京图表风格统一）
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.tick_params(axis='x', labelsize=10, rotation=30)
ax.tick_params(axis='y', labelsize=10)
ax.set_ylim(0, max(price_counts.values) * 1.2)  # 预留顶部空间显示标签

# ---------------------- 4. 保存图片（含深圳标识） ----------------------
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
save_path = os.path.join(os.getcwd(), f'shenzhen_price_distribution_{timestamp}.png')
plt.tight_layout()
plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
plt.close()
print(f"📁 Chart saved to: {save_path}")

✅ Loaded data: pet_funeral_shenzhen_cleaned_20251115_001632.xlsx (Total: 39 merchants in Shenzhen)
📁 Chart saved to: /Users/syx/Documents/python脚本/shenzhen_price_distribution_20251115_001756.png


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
from datetime import datetime

# ---------------------- 配色方案（深圳专属紫色系） ----------------------
colors = [
    '#f3e5f5',  # 最浅紫（<3.5）
    '#e1bee7',  # 浅紫（3.5-4.0）
    '#ce93d8',  # 中紫（4.0-4.5）
    '#9c27b0',  # 最深紫（4.5-5.0）
    '#e0d4e0'   # 淡紫灰（缺失数据）
]

# ---------------------- 1. 数据读取与预处理 ----------------------
# 筛选深圳最新清洗数据（文件前缀含"shenzhen_cleaned_"）
filtered_files = [f for f in os.listdir(os.getcwd()) if f.startswith("pet_funeral_shenzhen_cleaned_20251115_001632")]
if not filtered_files:
    print("❌ No Shenzhen filtered files found!")
    exit()
latest_file = max(filtered_files, key=lambda x: os.path.getmtime(x))
df = pd.read_excel(latest_file)
total = len(df)
print(f"✅ Loaded Shenzhen data: {latest_file} (Total: {total} merchants in Shenzhen)")

# 提取有效评分（逻辑与广州完全一致，确保数据处理统一）
def extract_valid_rating(star_val):
    if pd.isna(star_val) or str(star_val).strip() in ['0.0', '0.0分', '无评分', '', 'None']:
        return None
    match = re.search(r'(\d+\.?\d*)', str(star_val))
    return float(match.group()) if match and float(match.group()) > 0 else None

df['valid_rating'] = df['star'].apply(extract_valid_rating)
valid_data = df[df['valid_rating'].notna()]['valid_rating']
missing_count = total - len(valid_data)

# ---------------------- 2. 数据分组（与广州保持相同区间，便于对比） ----------------------
rating_bins = [0, 3.5, 4.0, 4.5, 5.0]  # 评分区间完全一致
rating_labels = ['<3.5(Poor)', '3.5-4.0(Fair)', '4.0-4.5(Good)', '4.5-5.0 (Excellent)', 'Missing Data']
rating_groups = pd.cut(valid_data, bins=rating_bins, labels=rating_labels[:-1], right=False)
rating_counts = rating_groups.value_counts().sort_index()
rating_counts['Missing Data'] = missing_count

# ---------------------- 3. 绘制饼图（布局与广州完全一致，仅替换配色和城市名） ----------------------
plt.style.use('seaborn-v0_8-white')
plt.rcParams['font.family'] = ['Arial', 'Helvetica']

fig, ax = plt.subplots(figsize=(9, 9))  # 与广州相同的正方形画布
wedgeprops = {'edgecolor': 'white', 'linewidth': 2}  # 白色边框区分扇区（广州同款样式）

wedges, texts, autotexts = ax.pie(
    rating_counts.values,
    labels=rating_labels,
    colors=colors,  # 应用深圳紫色系
    autopct='%1.1f%%',
    startangle=90,  # 起始角度与广州一致
    pctdistance=0.82,  # 百分比标签位置与广州相同
    labeldistance=1.1,  # 类别标签位置与广州相同
    wedgeprops=wedgeprops,
    textprops={'fontsize': 11}  # 文字大小统一
)

# 美化标签（与广州代码逻辑完全一致）
for autotext in autotexts:
    autotext.set_color('white')  # 白色百分比文字，与紫色系对比清晰
    autotext.set_fontweight('bold')
for text in texts:
    text.set_fontweight('bold')
    text.set_color('#4a148c')  # 深紫文字，与淡紫灰协调（对应广州的深棕文字逻辑）

# 标题（保持广州布局，仅替换城市名和配色关联元素）
ax.set_title(
    'Distribution of Pet Funeral Merchant Ratings in Shenzhen',
    fontsize=16, 
    fontweight='bold', 
    color='#9c27b0'  # 标题色与最深紫呼应（对应广州的最深绿标题）
)

# 图例分2列底部居中（与广州优化逻辑一致）
legend_labels = [f'{label} (n={int(rating_counts[i])})' for i, label in enumerate(rating_labels)]
ax.legend(
    legend_labels, 
    loc='center',
    bbox_to_anchor=(0.5, -0.1),  # 底部居中布局
    ncol=2,  # 分2列避免重叠
    fontsize=10,
    frameon=True,
    edgecolor='#e1bee7'  # 图例边框用浅紫（对应广州的浅绿边框）
)

# ---------------------- 4. 保存图片（文件名含深圳标识） ----------------------
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
save_path = os.path.join(os.getcwd(), f'shenzhen_rating_distribution_{timestamp}.png')
plt.tight_layout()
plt.savefig(save_path, dpi=300, bbox_inches='tight', facecolor='white')
plt.close()
print(f"📁 Shenzhen rating chart saved to: {save_path}")

✅ Loaded Shenzhen data: pet_funeral_shenzhen_cleaned_20251115_001632.xlsx (Total: 39 merchants in Shenzhen)


/var/folders/9q/ywtdr2kx78d4gg56pbn1dk_w0000gn/T/ipykernel_48324/182895037.py:81: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  legend_labels = [f'{label} (n={int(rating_counts[i])})' for i, label in enumerate(rating_labels)]


📁 Shenzhen rating chart saved to: /Users/syx/Documents/python脚本/shenzhen_rating_distribution_20251115_001833.png


In [5]:
%pip install -U pyecharts
%pip install echarts-china-cities-pypkg  # 安装最新城市地图数据

  Using cached pyecharts-2.0.9-py3-none-any.whl.metadata (1.6 kB)
Using cached pyecharts-2.0.9-py3-none-any.whl (153 kB)
  Attempting uninstall: pyecharts
    Found existing installation: pyecharts 1.9.1
    Uninstalling pyecharts-1.9.1:
      Successfully uninstalled pyecharts-1.9.1
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# 安装中国城市区级地图数据（包含深圳各行政区）
%pip install echarts-china-cities-pypkg --upgrade
# 若仍有问题，补充安装更细分的县级地图数据
%pip install echarts-china-counties-pypkg --upgrade

Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 4.6 MB/s eta 0:00:00a 0:00:01m
  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'echarts-china-counties-pypkg' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'echarts-china-counties-pypkg'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for echarts-china-counties-pypkg: filename=echarts_china_counties_pypkg-0.0.2-py3-none-any.whl size=6507407 sha256=deafbb372e18c6d0c65ba30bc69da527fd5ab4d31fc043cbb9b668296bbb24d4
  Stored in directory: /Users/syx/Library/Caches/pip/wheels/e3/72/99/c31d1bff045e08200d757eb4ba07758da5c4038f

In [29]:
import pandas as pd
import os
from pyecharts import options as opts
from pyecharts.charts import Map
from datetime import datetime

# ---------------------- 配置路径 ----------------------
INPUT_CLEANED_DATA = "/Users/syx/Documents/python脚本/shenzhen_second_cleaned_20251115.xlsx"  # 深圳清洗文件路径
OUTPUT_HEATMAP = f"深圳宠物店铺热力图_{datetime.now().strftime('%Y%m%d')}.html"


# ---------------------- 深圳所有行政区列表（含大鹏新区，确保完整） ----------------------
all_shenzhen_districts = [
    "南山区", "福田区", "罗湖区", "盐田区", "宝安区",
    "龙岗区", "龙华区", "坪山区", "光明区", "大鹏区"
]


# ---------------------- 生成热力图（深色紫色系+白色区名+正确分区） ----------------------
if os.path.exists(INPUT_CLEANED_DATA):
    df = pd.read_excel(INPUT_CLEANED_DATA)
    if "district_cn" in df.columns:
        district_counts = df["district_cn"].value_counts().to_dict()
        for dist in all_shenzhen_districts:
            if dist not in district_counts:
                district_counts[dist] = 0
        heatmap_data = [(k, v) for k, v in district_counts.items()]

        # 深紫色渐变配色（加深颜色，确保区名可见）
        purple_gradient = [
            "#f3e5f5", "#e1bee7", "#ce93d8", "#ba68c8",
            "#9c27b0", "#8e24aa", "#7b1fa2", "#4a148c"  # 最深色改为#4a148c，增强对比度
        ]

        c = (
            Map(
                init_opts=opts.InitOpts(
                    width="1200px",
                    height="700px",
                    page_title="Shenzhen Pet Funeral Heatmap"
                )
            )
            .add(
                series_name="Number of Merchants",
                data_pair=heatmap_data,
                maptype="深圳",  # 确保地图类型正确加载深圳全行政区
                label_opts=opts.LabelOpts(
                    is_show=True, 
                    font_size=10, 
                    color="black"  # 区名改为白色，深色区域也清晰可见
                ),
                itemstyle_opts=opts.ItemStyleOpts(
                    color=purple_gradient,  # 标记点颜色继承深紫色渐变
                    border_color="#fff",
                    border_width=1
                )
            )
            .set_global_opts(
                title_opts=opts.TitleOpts(
                    title="Distribution of Pet Funeral Stores in Shenzhen",
                    pos_left="center",
                    pos_top="15px",
                    title_textstyle_opts=opts.TextStyleOpts(
                        font_size=18, 
                        color="#4a148c",  # 标题色与最深紫呼应
                        font_weight="bold"
                    )
                ),
                legend_opts=opts.LegendOpts(
                    pos_bottom="10px",
                    pos_left="center",
                    textstyle_opts=opts.TextStyleOpts(color="#ba68c8")
                ),
                visualmap_opts=opts.VisualMapOpts(
                    min_=0,
                    max_=max([v for _, v in heatmap_data]) if heatmap_data else 1,
                    range_color=purple_gradient,
                    pos_right="20px",
                    pos_top="center",
                    orient="vertical",
                    is_calculable=True,
                    textstyle_opts=opts.TextStyleOpts(color="#ba68c8")
                ),
                toolbox_opts=opts.ToolboxOpts(
                    is_show=True,
                    feature={
                        "zoom": {"show": True, "title": "放大"},
                        "restore": {"show": True, "title": "重置"}
                    },
                    pos_right="30px",
                    pos_top="50px",
                    item_size=15
                )
            )
            .set_series_opts(
                tooltip_opts=opts.TooltipOpts(
                    formatter=lambda params: f"""
                    <div style="text-align:center; min-width:100px;">
                        <div style="font-weight:bold; color="#4a148c";>{params.name}</div>
                        <div style="color="#ba68c8";>{
                            "暂无店铺" if params.value == 0 
                            else f"Number of Merchants：{params.value}"
                        }</div>
                    </div>
                    """,
                    background_color="rgba(243, 229, 245, 0.9)",
                    border_color="#ce93d8",
                    textstyle_opts=opts.TextStyleOpts(color="#4a148c")
                )
            )
        )

        c.render(OUTPUT_HEATMAP)
        print(f"✅ 深圳热力图已保存：{OUTPUT_HEATMAP}")
    else:
        print("❌ 数据中无 'district_cn' 列，请检查清洗步骤")
else:
    print(f"❌ 未找到文件：{INPUT_CLEANED_DATA}")

✅ 深圳热力图已保存：深圳宠物店铺热力图_20251115.html
